Importandos todas as Bibliotecas necessárias.

In [1]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx2pdf import convert
import time
from num2words import num2words
import re
import os
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pyautogui
from selenium.webdriver.chrome.options import Options

Classe que realiza a busca de dados na planilha do Excel e retorna um dicionário:<br>
 
 * Arquivo do Excel com uma tabela por aba;

 * Se não for especificada a aba, ele irá buscar em todas as abas até encontrar a primeira ocorrência;

 * Se a célula for vazia, a mesma será excluído do dicionário;
 
 * Importante analisar a ordem das abas, abas com valores únicos devem ser as primeiras da planilha;

 * Se na célular houver valores separados por ';' será criada uma lista com os valores separados pelo ';'.

In [4]:
class BuscarDados:

    def __init__(self, valor, aba=None):
        
        if not aba:
            banco_de_dados = pd.read_excel('01_Cadastro.xlsm', sheet_name=None)
            for dados_aba in banco_de_dados.values():
                if valor in dados_aba.values:
                    banco_de_dados = dados_aba
                    break
            else:
                raise ValueError('O valor não foi encontrado nas abas do arquivo')

        else:
            banco_de_dados = pd.read_excel('01_Cadastro.xlsm', sheet_name = aba)
            if valor not in banco_de_dados.values:
                raise ValueError('O valor não foi encontrado nas abas do arquivo')

        linha = banco_de_dados.loc[banco_de_dados.isin([valor]).any(axis = 1)].iloc[0].dropna().astype(str)
        self.dicionario = {indice: valor.split(';') if ';' in valor
                            else valor for indice, valor in linha.items()}
        
        if 'ATRIBUTOS' in self.dicionario:            
            if isinstance(self.dicionario['ATRIBUTOS'], str):
                self.dicionario[self.dicionario.pop('ATRIBUTOS')] = self.dicionario.pop('VALORES')
            else:
                self.dicionario.update(dict(zip(self.dicionario.pop('ATRIBUTOS'), self.dicionario.pop('VALORES'))))

Classe que cria um dicionária da Parte procurada:
* As chaves do dicionário são os atributos da parte, com seus valores como os valores;

* Qualquer atributo pode ser usado para buscar a parte (nome, cpf, cnpj, etc...).

In [5]:
class QualificarParte:
    
    def __init__(self, parte):
        
        self.dicionario = BuscarDados(parte).dicionario
        self._agrupar_endereco()
        self._qualificar_parte()
          
    def _agrupar_endereco(self):
        
        self.dicionario['NUMERO_COMPLEMENTO'] = (f"{self.dicionario['NUMERO']} - {self.dicionario['COMPLEMENTO']}"
                                                if 'COMPLEMENTO' in self.dicionario else self.dicionario['NUMERO'])
                
        self.dicionario['ENDERECO'] = '{}, nº {}, Bairro {}, CEP {}, {}/{}'.format(
                                        self.dicionario['LOGRADOURO'], self.dicionario['NUMERO_COMPLEMENTO'],
                                        self.dicionario['BAIRRO'], self.dicionario['CEP'],
                                        self.dicionario['CIDADE'], self.dicionario['UF']
                                        )
   
    def _qualificar_parte(self):

        if 'CPF' in self.dicionario:
            self.dicionario['QUALIFICAÇÃO'] = '{}, {}, {}, {}, RG n° {} {}, CPF n° {}, e-mail: {}, com endereço à {}'.format(
                                                self.dicionario['NOME'], self.dicionario['NACIONALIDADE'], self.dicionario['ESTADO_CIVIL'],
                                                self.dicionario['PROFISSAO'], self.dicionario['RG'], self.dicionario['EXPEDIDOR_RG'],
                                                self.dicionario['CPF'], self.dicionario['EMAIL'],self.dicionario['ENDERECO']
                                                )

        elif 'TAG' in self.dicionario and self.dicionario['TAG'] == 'condominio':
            self.dicionario['QUALIFICAÇÃO'] = ('{}, inscrito no CNPJ n° {}, com endereço à {}, devidamente representado neste ato '
                                               'pelo seu síndico(a) {}'.format(
                                                self.dicionario['NOME'], self.dicionario['CNPJ'],
                                                self.dicionario['ENDERECO'], self.dicionario['ADMINISTRADOR'])
                                                )
            
        else:
            self.dicionario['QUALIFICAÇÃO'] = ('{}, pessoa jurídica inscrita no CNPJ n° {}, com endereço à {}, devidamente '
                                                'representada neste ato pelo(a) sócio(a) {}').format(
                                                self.dicionario['NOME'], self.dicionario['CNPJ'],
                                                self.dicionario['ENDERECO'], self.dicionario['ADMINISTRADOR']
                                                )

Classe que cria um dicionário do documento procurado:
* O documento deve ser procurado pelo ID (pois é o atributo único dele);

* O cliente ou o adverso podem ser mais de uma pessoa;

* O adverso é opcional.

In [6]:
class CriarDocumento:
    
    def __init__(self, id_documento):
        
        self.dicionario = BuscarDados(id_documento, 'Peticoes').dicionario
        self._dicionario_parte(self.dicionario['CLIENTE'])
        if 'ADVERSO' in self.dicionario:
            self._dicionario_parte(self.dicionario['ADVERSO'], 'ADVERSO_')
        
        if self.dicionario['TIPO'] == 'Peticao':
            dados_do_processo = BuscarDados(self.dicionario['PROCESSO'], 'Processos')
            self.dicionario['VARA'] = dados_do_processo.dicionario['VARA']
            self.dicionario['COMARCA'] = dados_do_processo.dicionario['COMARCA']

    def _dicionario_parte(self, parte_do_documento, prefixo = 'CLIENTE_'):
        
        def dicionario_se_parte_unica(parte_do_documento, prefixo):
            for chave, valor in QualificarParte(parte_do_documento).dicionario.items():
                nova_chave = prefixo + chave
                self.dicionario[nova_chave] = valor
        
        # Checa o número de parte se é multiplo (lista).
        if isinstance(parte_do_documento, str):
            dicionario_se_parte_unica(parte_do_documento, prefixo)
        
        else:
            dicionario_se_parte_unica(parte_do_documento[0], prefixo)
            self.dicionario[prefixo+'QUALIFICAÇÃO'] = f"(1){self.dicionario[prefixo+'QUALIFICAÇÃO']}"
            self.dicionario['(1)'+prefixo+'QUALIFICAÇÃO'] = self.dicionario[prefixo+'QUALIFICAÇÃO']
            
            for index, nome in enumerate(parte_do_documento[1:], start=2):
                self.dicionario[prefixo+'QUALIFICAÇÃO'] += f" e ({index}){QualificarParte(nome).dicionario['QUALIFICAÇÃO']}"
                prefixo2 = f'({index})' + prefixo
                dicionario_se_parte_unica(nome, prefixo2)

Salva o arquivo em docx e pdf:

* Substitui os placeholders(chaves) de um arquivo docx pelos valores contidos nas chaves do dicionário criado com 'CriarDocumento';<br>

* Formata o nome do arquivo de acordo com padronizado;

* Na sequência, salva em um arquivo docx em uma pasta já especificada e converte também em pdf.

In [35]:
class CriarDocxPdf:

    def __init__(self, id_documento):
        
        dicionario = CriarDocumento(id_documento).dicionario
        documento = Document(f"01_Modelos\{dicionario['PECA']}.docx")
        self._substituir_texto(dicionario, documento)      

        nome_arquivo = self._formatar_nome_arquivo(dicionario)
        documento.save(f'02_Documentos-prontos\{nome_arquivo}.docx')
        convert(f'02_Documentos-prontos\{nome_arquivo}.docx', f'02_Documentos-prontos\{nome_arquivo}.pdf')

    def _substituir_texto(self, dicionario, docx):

        def numero_por_extenso(paragrafo):
            
            numero_entre_parenteses = r"\((.*?)\)"
            resultado_busca_numero = re.search(numero_entre_parenteses, paragrafo.text)
            if resultado_busca_numero:
                valor_encontrado = resultado_busca_numero.group(1)
                valor_decimal = valor_encontrado.replace('.', '').replace(',', '.')
                try:
                    valor_por_extenso = '(' + num2words(float(valor_decimal), lang='pt_BR', to='currency') + ')'
                    paragrafo.text = paragrafo.text.replace(resultado_busca_numero.group(0), valor_por_extenso)
                except ValueError:
                    pass

        dicionario_de_substituicao = {'x' + chave.upper() + 'x': valor for chave, valor in dicionario.items()}
        
        for paragrafo in docx.paragraphs:
            for chave in dicionario_de_substituicao:
                if chave in paragrafo.text:
                    paragrafo.text = paragrafo.text.replace(chave, dicionario_de_substituicao[chave])
            numero_por_extenso(paragrafo)

    def _formatar_nome_arquivo(self, dicionario):
        
        peca = dicionario['PECA'] + '_'
        parte = (dicionario['CLIENTE_NOME'].capitalize() + '-x-' + dicionario['ADVERSO_NOME'].lower()
                 if 'ADVERSO_NOME' in dicionario else dicionario['CLIENTE_NOME'].capitalize())
        return unidecode((peca + parte).replace(' ','-'))

In [ ]:
CriarDocxPdf(2)

In [31]:
class EmitirCustasMinimas:

    def __init__(self, id_documento):
        dicionario = CriarDocumento(id_documento).dicionario        
        driver = webdriver.Chrome()

        
    def emitir_ar(self):
        driver.get('https://www45.bb.com.br/fmc/frm/fw0707314_1.jsp')
        input_nome = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[1]/td/table[1]/tbody/tr[2]/td[1]/input'
        input_cpf = '//*[@id="cpf"]'
        input_processo = '//*[@id="num_processo"]'
        input_unidade = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[1]/td/table[2]/tbody/tr[2]/td[2]/input'
        input_historico = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[1]/td[1]/table/tbody/tr[2]/td/textarea'
        input_valor = '//*[@id="valor"]'
        
        input_codigo = '//*[@id="cod"]'
        input_postais = '//*[@id="showTable"]/div/table/tbody/tr/td/table[2]/tbody/tr/td[2]/table[2]/tbody/tr[1]/td[4]/a'
        gerar_guia = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[5]/tbody/tr/td/input[1]'
        imprimir = ''

    #def fecharsite():
     #   driver.quit() 

In [89]:
driver = webdriver.Chrome()

driver.get('https://www45.bb.com.br/fmc/frm/fw0707314_1.jsp')

input_nome = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[1]/td/table[1]/tbody/tr[2]/td[1]/input'
driver.find_element(By.XPATH, input_nome).send_keys('joao joa jjo')

input_cpf = '//*[@id="cpf"]'
driver.find_element(By.XPATH, input_cpf).send_keys('01736927027')

input_processo = '//*[@id="num_processo"]'
driver.find_element(By.XPATH, input_processo).send_keys('11')

input_unidade = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[1]/td/table[2]/tbody/tr[2]/td[2]/input'
driver.find_element(By.XPATH, input_unidade).send_keys('V. Civel')

input_historico = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[1]/td[1]/table/tbody/tr[2]/td/textarea'
driver.find_element(By.XPATH, input_historico).send_keys('AR citação')

input_valor = '//*[@id="valor"]'
driver.find_element(By.XPATH, input_valor).send_keys('2970')

input_codigo = '//*[@id="cod"]'
driver.find_element(By.XPATH, input_codigo).click()

input_postais = '//*[@id="showTable"]/div/table/tbody/tr/td/table[2]/tbody/tr/td[2]/table[2]/tbody/tr[1]/td[4]/a'
driver.find_element(By.XPATH, input_postais).click()

gerar_guia = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[5]/tbody/tr/td/input[1]'
driver.find_element(By.XPATH, gerar_guia).click()

driver.execute_script('window.print();')
time.sleep(3)

pyautogui.press('enter')
time.sleep(3)

driver.quit()



In [93]:


site2 = 'https://www45.bb.com.br/fmc/frm/fw0707314_1.jsp'
chrome_options = Options()
chrome_options.add_argument('--kiosk-printing')  # Modo de impressão em tela cheia
driver = webdriver.Chrome(options=chrome_options)
driver.get(site2)

# Seu código para preencher os campos e gerar a guia
input_nome = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[1]/td/table[1]/tbody/tr[2]/td[1]/input'
driver.find_element(By.XPATH, input_nome).send_keys('joao joa jjo')

input_cpf = '//*[@id="cpf"]'
driver.find_element(By.XPATH, input_cpf).send_keys('01736927027')

input_processo = '//*[@id="num_processo"]'
driver.find_element(By.XPATH, input_processo).send_keys('11')

input_unidade = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[1]/td/table[2]/tbody/tr[2]/td[2]/input'
driver.find_element(By.XPATH, input_unidade).send_keys('V. Civel')

input_historico = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[1]/td[1]/table/tbody/tr[2]/td/textarea'
driver.find_element(By.XPATH, input_historico).send_keys('AR citação')

input_valor = '//*[@id="valor"]'
driver.find_element(By.XPATH, input_valor).send_keys('2970')

input_codigo = '//*[@id="cod"]'
driver.find_element(By.XPATH, input_codigo).click()

input_postais = '//*[@id="showTable"]/div/table/tbody/tr/td/table[2]/tbody/tr/td[2]/table[2]/tbody/tr[1]/td[4]/a'
driver.find_element(By.XPATH, input_postais).click()

gerar_guia = '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[5]/tbody/tr/td/input[1]'
driver.find_element(By.XPATH, gerar_guia).click()

#driver.execute_script('window.print();')
# ... preencha os outros campos ...
# Aguarde alguns segundos para garantir que a página seja carregada completamente
time.sleep(3)

# Abre o menu de impressão com Ctrl + P (pode variar dependendo do sistema operacional)
driver.execute_script('window.print();')

# Aguarde mais alguns segundos para a caixa de diálogo de impressão aparecer
time.sleep(3)

# Pressione a tecla 'Enter' para confirmar a impressão
driver.switch_to.alert.accept()

# Aguarde até que o download do PDF seja concluído (pode variar dependendo do tamanho do PDF)
time.sleep(3)

# Feche o navegador
driver.quit()



NoAlertPresentException: Message: no such alert
  (Session info: chrome=115.0.5790.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0106A813+48355]
	(No symbol) [0x00FFC4B1]
	(No symbol) [0x00F05220]
	(No symbol) [0x00EFE5DF]
	(No symbol) [0x00F18824]
	(No symbol) [0x00F4A7B4]
	(No symbol) [0x00F186ED]
	(No symbol) [0x00F4A536]
	(No symbol) [0x00F282DC]
	(No symbol) [0x00F293DD]
	GetHandleVerifier [0x012CAABD+2539405]
	GetHandleVerifier [0x0130A78F+2800735]
	GetHandleVerifier [0x0130456C+2775612]
	GetHandleVerifier [0x010F51E0+616112]
	(No symbol) [0x01005F8C]
	(No symbol) [0x01002328]
	(No symbol) [0x0100240B]
	(No symbol) [0x00FF4FF7]
	BaseThreadInitThunk [0x75B700C9+25]
	RtlGetAppContainerNamedObjectPath [0x772B7B1E+286]
	RtlGetAppContainerNamedObjectPath [0x772B7AEE+238]


In [22]:


time.sleep(1)

preencher_nome = driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[1]/tbody/tr[1]/td/table[1]/tbody/tr[2]/td[1]/input')

texto_nome = 'Texto de exemplo'
preencher_nome.send_keys(texto_para_preencher)


pdf_path = r'\02_Documentos-prontos\arquivo.pdf' 
driver.execute_script('window.print();')

botao = driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/form/table/tbody/tr/td[2]/table[5]/tbody/tr/td/input[1]')

#botao.click()

time.sleep(3)

# Fechar o navegador
driver.quit()

In [11]:
import pyodbc

dados_conexao = ('Driver={SQL Server};Server=DESKTOP-SMFGJPJ\SQLEXPRESS;Database=teste')
conexao = pyodbc.connect(dados_conexao)
print('Conexão Bem sucedida')
cursor = conexao.cursor()
cursor.execute("SELECT * FROM nome")
cursor.commit()
cursor.close()
conexao.close()


Conexão Bem sucedida


In [ ]:
cursor.execute("INSERT INTO tabela_origem (campo1, campo2) VALUES (?, ?)", valor1, valor2)

# Obter o ID do último registro inserido
cursor.execute("SELECT SCOPE_IDENTITY() AS LastID")
id_origem = cursor.fetchone().LastID

# Inserir dados na tabela de destino usando o ID da tabela de origem
cursor.execute("INSERT INTO tabela_destino (id_origem, campo3, campo4) VALUES (?, ?, ?)", id_origem, valor3, valor4)

# Confirmar as alterações
cursor.commit()


